# Model for recomendations solutions on drainage.

## 1. Import libraries

In [1]:
import swmmio
import pyswmm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental import preprocessing


desired_width = 500
pd.set_option("display.width", desired_width)
np.set_printoptions(linewidth=desired_width)
pd.set_option("display.max_columns", 30)

## 2. Recomendation classes

In [2]:
classes = pd.DataFrame(
    data={
        "classes": [
            "pump",
            "tank",
            "seepage_boxes",
            "diameter_increase",
        ]
    }
)
classes

,classes
0,pump
1,tank
2,seepage_boxes
3,diameter_increase


## 3. Get data

In [ ]:
INP_FILE = "dataset/subcatchment_dataset.inp"

### Run simulation

In [ ]:
with pyswmm.Simulation(INP_FILE) as sim:
    for step in sim:
        pass

### Read inp and rpt file as swmmio model object

model = swmmio.Model(INP_FILE)

### Get data from the swmmio model

In [4]:
# TODO
data = ""

### Drop unused columns

In [5]:
data.drop(['fill columns name'], axis=1, inplace=True)

AttributeError: 'str' object has no attribute 'drop'